In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# GitHub URL for dataset
base_url = "https://github.com/chascc/DL-Model-Training/tree/main/data/natural_images"

# Data generators (adjust the image size and batch size as before)
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
load_train = data_gen.flow_from_directory(
    base_url,
    new_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
load_val = data_gen.flow_from_directory(
    base_url,
    new_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


In [ ]:
# Displays class labels
class_name = list(load_train.class_indices.keys())
print("Class Labels:", class_name)

# Displays example images
plt.figure(figsize=(10, 5))
for images, labels in load_train:
    # Displays 6 example images
    for i in range(6): 
        plt.subplot(2, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_name[np.argmax(labels[i])])
        plt.axis('off')
    break
plt.tight_layout()
plt.show()

In [ ]:
# Initializes the CNN model using tensorflow.keras library
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_h, img_w, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Output layer uses softmax activation
    Dense(len(class_name), activation='softmax') 
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)
model.save('natural_images_cnn_model.keras')


In [ ]:
# Evaluates the model
loss, accuracy = model.evaluate(load_val)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
# Plots the training history
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# Predict some images from validation data
import random

# Gets a batch of images from the folder at random
images, labels = next(load_val)
sample_indices = random.sample(range(len(images)), 6)

plt.figure(figsize=(10, 5))
for i, idx in enumerate(sample_indices):
    plt.subplot(2, 3, i + 1)
    plt.imshow(images[idx])
    predicted_class = class_name[np.argmax(model.predict(images[idx][np.newaxis, ...]))]
    true_class = class_name[np.argmax(labels[idx])]
    plt.title(f"True: {true_class}\nPred: {predicted_class}")
    plt.axis('off')
plt.tight_layout()
plt.show()